<a style="padding-left:600px" href="https://colab.research.google.com/github/emasri/trd/blob/master/td.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Internet speed

In [ ]:
#@markdown
!sudo apt install speedtest-cli
!speedtest-cli

#### Start drive

In [ ]:
#@markdown
import enum
import os, time,  re, IPython, urllib
os.environ['TZ'] = 'Turkey'
time.tzset()
drv = next(os.walk("/content"))[1]
if not "drive" in drv:
  from google.colab import drive
  drive.mount('/content/drive')
#from os import makedirs makedirs("/root/.config/rclone", exist_ok = True)

def auto_reconnect(): 
  from google.colab import output
  display(IPython.display.Javascript('''function ClickConnect(){btn = document.querySelector("colab-connect-button") if (btn != null){console.log("Click colab-connect-button"); btn.click() } btn = document.getElementById('ok') if (btn != null){console.log("Click reconnect"); btn.click() }} setInterval(ClickConnect,60000)'''))

def dump(obj):
  for attr in dir(obj):
    print("                   %s =   %r" % (attr, getattr(obj, attr)))


Save_Base = "/content/drive/Shareddrives/SM1/Downloads"  #@param {type:"string"}
Movie_Path = "/content/drive/Shareddrives/SM1/Library/Movies" #@param {type:"string"}
Shows_Path = "/content/drive/Shareddrives/SM1/Library/TV Shows" #@param {type:"string"}
Part = "1" #@param {type:"string"}

drive_path = "" 
done_path = "/content/done"

Debug = False #@param {type:"boolean"}
if Debug:
  drive_path = "/content/debug"
else:
  drive_path = Save_Base + "/d-0" + Part

temp_save="/content/d-0" + Part
os.makedirs("/content/done", exist_ok = True)
os.makedirs(temp_save, exist_ok = True)
os.makedirs("/content/Added", exist_ok = True)

#@markdown 
Test_drive = False #@param {type:"boolean"}
if Test_drive:
  direct_url = "https://download.visualstudio.microsoft.com/download/pr/c4c86d02-a47b-4bd4-b73d-ec3be19e5245/76c673e22a120464c95f85bef342a361/aspnetcore-runtime-6.0.7-win-x64.exe" 
  output_dir = "/content/test" 
  !wget "$direct_url" -O "$output_dir"  
  !mv "/content/test" "/content/drive/My Drive"


print(" Local temporary saving path  " + temp_save)
print(" Local done saving path  " + done_path)
print(" Drive main saving path:  " + drive_path)
print(" Drive movie saving path:  " + Movie_Path)
print(" Drive show saving path:  " + Shows_Path)

def filter_dirs(ps):
  for p in ps:
    if p[0] == '.':
      ps.remove(p)
  return ps


def print_files(drs, range_start , range_end):
  for i in range(range_start, range_end):
    print("   {0}-  {1}".format(i+1, drs[i])) 
    

def RoundTo1(n) :
  if n < 1 and (n-0.9765625) >= 0 :
    return 1
  else: 
    return n

def data_str(byte_val):
  size_gb = ByteToGB(byte_val)
  if (size_gb >= 1) :
    return "{:.1f} GB".format(size_gb)
  else:
    size_mb = ByteToMB(byte_val)
    return "{:.1f} MB".format(size_mb)

def speed_str(byte_val):
  size_mb = ByteToMB(byte_val)
  if (size_mb >= 1) :
    return "{:.1f} MB".format(size_mb)
  else:
    return "{:.1f} KB".format(ByteToKB(byte_val))

ByteToGB = lambda n: RoundTo1(n / 1073741824)
ByteToMB = lambda n: RoundTo1(n/ 1048576)
ByteToKB = lambda n: RoundTo1(n/ 1024)

downloads = []
completed = []
active=[]

rx = r"\d{4}"

def new_tv(name): 
  name = name.strip()
  table = name.maketrans("."," ","'()")
  trans = name.translate(table)
  dgts = re.findall(rx, trans)
  if '1080' in dgts:
    dgts.remove('1080')
  if dgts:  
    return trans.split(dgts[0])[0] + dgts[0]
  else:
    return trans

def new_mv(name): 
  name = name.strip()
  table = name.maketrans("."," ","'()")
  trans = name.translate(table)
  digits = re.findall(rx, trans)
  if '1080' in digits:
    digits.remove('1080')
  if len(digits) >= 2:
       return trans.split(digits[1])[0] + digits[1]   
  elif digits:  
      return trans.split(digits[0])[0] + digits[0]
  else:
    return trans

def movie_name_trans(name): 
  name = name.strip().replace("&","and")
  dgts = re.findall(rx, name)
  if '1080' in dgts:
    dgts.remove('1080')
  if dgts:  
    if len(dgts) == 1:
      name = name.split(dgts[0])[0]
    else:
      name = name.split(dgts[1])[0] 
  table = name.maketrans("",""," '(#$%-,.;:!?)")
  trans = name.translate(table).lower()
  return trans

def show_name_trans(name): 
  name = name.strip()
  dgts = re.findall(rx, name)
  if dgts:
    name = name.split(dgts[0])[0].lower()
  else :
    name = name.lower()
  table = name.maketrans("",""," '(#$%&-,.;:!?)")
  trans = name.translate(table)
  return trans

def movie_exists(name):
  name_s = new_mv(name)
  path = Movie_Path + "/" + name_s
  return os.path.exists(path)

number =  1 
number2 = 0.9534856384568
number3= round(number2,2)
str1 = "text"
num_str = str(number2)
  # movies_dir = filter_dirs(next(os.walk(Movie_Path))[1])
  # movies = [movie_name_trans(x) for x in movies_dir]
  # name_s = movie_name_trans(name)
  # if name_s in movies :
  #   return True
  # else:
  #   return False

def show_exists(name):
  name_s = new_tv(name)
  path = Shows_Path + "/" + name_s
  return os.path.exists(path)

  # shows_dir = filter_dirs(next(os.walk(Shows_Path))[1])
  # shows = [show_name_trans(x) for x in shows_dir]
  # name_s = show_name_trans(name)
  # if name_s in shows :
  #   return True
  # else:
  #   return False

def get_magnet_name(magnet):
  return urllib.parse.unquote(magnet.split("dn=")[1].split("&tr=")[0], encoding='utf-8', errors='replace').replace('+',".")

skip_list = ['RARBG.txt',
             'RARBG_DO_NOT_MIRROR.exe',
             'www.YTS.LT.jpg',
             '.txt',
              '.inf']

def add_torrent_file(ti, type):
  downloads = ses.get_torrents()
  name =ti.name()
  print("Downloading: " + name)
  params = {
  "save_path": temp_save,
  "ti": ti}
  print("Files: {0}".format(ti.num_files()))
  print("Metadata size: {0}".format(speed_str(ti.metadata_size())))
  print("Download files:\n")
  files = ti.files()
  handle = ses.add_torrent(params)
  download_checks.append(type)
  if handle in downloads:
    print(" Torrent already added.\n")
    return
  handle.auto_managed(True)
  active.append(handle)

  for index in range(files.num_files()):
    if any(i in files.file_name(index) for i in skip_list):
      handle.file_priority(index,0)
    else: 
      handle.file_priority(index,6)
      fs_str= data_str(files.file_size(index))
      fi_str = "   {:3d}-  ".format(index) + fs_str
      print(fi_str, files.file_name(index))
  print()
  
def add_torrent_magnet(magnet_link, name):
  params = {"save_path": temp_save}
  handle = lt.add_magnet_uri(ses, magnet_link, params)    
  if not handle.is_valid():
    print(" Invalid torrent  {0}".format(name))
    return
  elif handle in active:
    print(" Torrent already added.\n")
    return
  else :
    handle.auto_managed(True)
    active.append(handle)
    if handle.has_metadata():
      ti = handle.get_torrent_info()
      files = ti.files()
      print()
      for index in range(files.num_files()):
        if any(i in files.file_name(index) for i in skip_list):
          handle.file_priority(index,0)
        else: 
          handle.file_priority(index,6)
          fs_str= data_str(files.file_size(index))
          fi_str = "   {:3d}-  ".format(index) + fs_str
          print(fi_str, files.file_name(index))
  print()

def add_local_files(type):
  print("Loading torrent files in /content directory:\n")
  fls = next(os.walk("."))[2]
  for fl in fls:
    splited = os.path.splitext(fl)
    if splited[1] == ".torrent":
      ti = lt.torrent_info(fl)
      name =ti.name()
      if not ti.is_valid():
        print("Invalid torrent.")
        continue
      if type == check_types.mv and movie_exists(name):
        print(name + " is already downloaded.")
        continue
      elif type == check_types.season and show_exists(name):
        print(name + " is already downloaded.")
        continue
      elif type == check_types.episode and show_exists(name):
        print(name + " is already downloaded.")
        continue
      add_torrent_file(ti, type)
      !mv "$fl" "/content/Added"

def shave (handle, savepath) :
  ti = handle.get_torrent_info()
  files = ti.files()
  priorities = handle.file_priorities();
  for i in range(len(priorities)):
    path = savepath + "/" + files.file_path(i)
    if (priorities[i] == 0 or files.file_name(i) in skip_list) and os.path.isfile(path):
      !rm "$path"

def after_shave (savepath, newname) :
  dircontent = next(os.walk(savepath))[2]
  for file in dircontent:
    split = os.path.splitext(file)
    ext = split[1]
    if  (ext == ".mp4" or ext == ".mkv") and not split[0] == newname:
      p1 = savepath + "/" + file
      p2 = savepath + "/" + newname + ext
      !mv "$p1" "$p2"

    elif file in skip_list:
      p0 = savepath + "/" + file
      !rm "$p0"

    subspath = savepath + "/" + "Subs"

    if not os.path.exists(subspath):
      subspath = savepath + "/" + "subs"
      if not os.path.exists(subspath):
        return

    subs =  next(os.walk(subspath))[2]
    count = 0
    extra = ""
    for sub in subs:
      split = os.path.splitext(sub)
      ext = split[1]
      subname = newname + extra
      if ext == '.srt' and split[0] != subname:
        if count > 0:
          extra = str(count)
        p1 = subspath + "/" + sub
        p2 = subspath + "/" + subname + ext
        !mv "$p1" "$p2"
        count+=1

def clean_tv (path):
  dircontent = next(os.walk(path))[1]
  for season in dircontent:
    seasonpath = path + "/" + season
    clean_season(seasonpath)

def clean_season(path , season):
  if len(season) > 3:
    checkname = season.replace(" ","").lower()
    for i in range(len(seasons)):
      if seasons[i] in season:
        s0 = "S0"+ str(i+1)
        p1 = path + "/" + s0
        !mv "$seasonpath" "$s0"
        path = p1
        if seasons[i] != seasons[-1]:
          temp = seasons[0]
          seasons[0] = seasons[i + 1]
          seasons[i + 1] = temp
        break

  files = next(os.walk(path))[2]
  for file in files:
    splitted = os.path.splitext(file)
    name = splitted[0]
    ext = splitted[1]
    if "." in name and (ext == ".mkv" or ext == ".mp4" or ext == ".srt"):
      newname = new_episode(name) + ext
      p1 = path + "/" + file
      p2 = path + "/" + newname
      !mv "$p1" "$p2"  

def new_episode(name): 
  name = name.strip()
  table = name.maketrans("."," ","'()")
  trans = name.translate(table)
  return trans.split("1080")[0]  

seasons = ["season1", "season2", "season3", "season4", "season5", "season6", "season7", "season8", "season9", "season", "season10", "season11", "season12", "season13", "season14",  ] 
download_checks = []
class check_types (enum.Enum):
  general = 1 
  mv = 2
  tv = 3
  season = 4
  episode = 5

auto_reconnect()

#### Start Session

In [ ]:
#@markdown
 
!python -m pip install --upgrade pip setuptools wheel &> /dev/null
!python -m pip install lbry-libtorrent &> /dev/null
!apt install python3-libtorrent &> /dev/null

!pip install rarbgapi &> /dev/null
!pip3 install requests &> /dev/null
!pip3 install beautifulsoup4 &> /dev/null

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)

import requests
from bs4 import BeautifulSoup

watched_movies_list = []
watched_list_url = "https://www.imdb.com/search/title/?title_type=feature&lists=ls047519430&view=simple&sort=alpha,asc&count=250&start="
page1 = watched_list_url + "0"
page2= watched_list_url + "251"


def scrap_watched_list(text):
    soup = BeautifulSoup(text,'html.parser')    
    scraped = soup.find("div",{"class":"lister-list"})
    if not scraped :
        return
    all_items = scraped.findAll(class_="lister-item-header")
    for i in range(len(all_items)):
        year = all_items[i].findChild(class_="lister-item-year")
        title = "{0} {1}".format(all_items[i].a.text, year.text).replace("(I) ", "").replace(": ", " ").replace(". ",".")
        table = title.maketrans("."," ","(,'?)")
        title = title.translate(table)
        watched_movies_list.append(title)
    return

reponse1 = requests.get(page1, headers = {"Accept-Language": "en-US"})
scrap_watched_list(reponse1.text)
reponse2 = requests.get(page2, headers = {"Accept-Language": "en-US"})
scrap_watched_list(reponse2.text)


#### Settings

In [ ]:
#@markdown

Announce_To_All_Tiers = True #@param {type:"boolean"}
Announce_To_All_Trackers = True #@param {type:"boolean"}
Active_Limit = 5 #@param {type:"integer"}
Active_Downloads = 3 #@param {type:"integer"}
Active_Seeds = 1 #@param {type:"integer"}
Active_Checking = 3 #@param {type:"integer"}
Connections_Limit = 2330 #@param {type:"integer"}
Connection_Speed = 80 #@param {type:"integer"}
Connections_Slack = 50 #@param {type:"integer"}
File_Pool_Size = 20 #@param {type:"integer"}
Half_Open_Limit = 100 #@param {type:"integer"}
Max_Rejects = 8 #@param {type:"integer"}
Max_Peerlist_Size = 5000 #@param {type:"integer"}
Max_Paused_Peerlist_Size = 2000 #@param {type:"integer"}
Max_Failcount = 3 #@param {type:"integer"}
Min_Announce_Interval = 30 #@param {type:"integer"}
Min_Reconnect_Time = 20 #@param {type:"integer"}
Request_Timeout = 10 #@param {type:"integer"}
Smooth_Connects = False #@param {type:"boolean"}
Send_Buffer_Watermark_MB = 5  #@param {type:"integer"}
Seed_Time_Limit = 60 #@param {type:"integer"}
Tracker_Completion_Timeout = 30 #@param {type:"integer"}
Upload_Rate_Limit_KB = 0 #@param {type:"integer"}
User_Agent = '"Utorrent"/3.5.5' #@param {type:"string"}

sett = {
    'allow_multiple_connections_per_ip': True,
    'active_downloads': Active_Downloads,
    'active_seeds':Active_Seeds,
    'active_checking': Active_Checking,
    'active_limit':Active_Limit,
    'active_seeds':Active_Seeds,    
    'announce_to_all_tiers': Announce_To_All_Tiers,
    'announce_to_all_trackers':Announce_To_All_Trackers,
    'allow_multiple_connections_per_ip':True,
    'auto_sequential':True,
    'auto_manage_interval': 30,
    'connections_limit':Connections_Limit,
    'connection_speed': Connection_Speed,
    'connections_slack':Connections_Slack,
    'close_redundant_connections':True,
    'file_pool_size':File_Pool_Size,
    'local_service_announce_interval':200,
    'max_rejects':Max_Rejects,
    'max_peerlist_size': Max_Peerlist_Size,
    'max_paused_peerlist_size': Max_Paused_Peerlist_Size,
    'max_failcount':Max_Failcount,
    'min_announce_interval':Min_Announce_Interval,
    'min_reconnect_time': Min_Reconnect_Time,
    'peer_timeout':20,
    'request_timeout':Request_Timeout,
    'half_open_limit': Half_Open_Limit,
    'send_buffer_watermark':Send_Buffer_Watermark_MB*1024*1024,
    'smooth_connects':Smooth_Connects,
    'strict_end_game_mode':True,
    'seed_time_limit': Seed_Time_Limit,
    'tracker_completion_timeout':Tracker_Completion_Timeout,
    'upload_rate_limit':Upload_Rate_Limit_KB*1024,
    'user_agent': User_Agent,
    #'inactivity_timeout':20,
    #'ban_web_seeds':False,
    }

ses.apply_settings(sett)

#'inactivity_timeout':20,
#'ban_web_seeds':False,
  

#### Load uncompleted

In [ ]:
#@markdown 

Load_From = "/content/drive/Shareddrives/SM1/Downloads/.uncomplete" #@param {type: "string"}

content = next(os.walk(Load_From))[1]

if content:
  print(" Choose a directory to load:")
  for i,c in enumerate(content[:]):
    print("   {0}-  {1}".format(i+1, c))

  print()
  choice = int(input(""))-1
  print()
  if(choice >= len(content) or choice < 0):
    print("The choice is out of range.")
  else:
    print("Loading {0} into {1}".format(content[choice], temp_save))
    print()
    path = Load_From+"/"+content[choice]
    !cp -r "$path" "$temp_save"
    print("   ...................")
    print("          Done")

#### Review settings

In [ ]:
#@markdown

settings = ses.get_settings()
setn =[]

for x in settings:
  setn.append(str(x))

setn.sort()

for x in setn:
  print("'{0}': {1},".format(x,settings[x]))

#### Search

In [ ]:
import requests
from bs4 import BeautifulSoup 
Movie = False #@param {type: "boolean"}
TV_Show = False #@param {type:"boolean"}
TV_Season = False #@param {type:"boolean"}
TV_Episode = False  #@param {type:"boolean"}
#@markdown 

Search = "" #@param {type:"string"}
Page =  1#@param {type:"integer"}

#@markdown 

By_Seeders = False #@param {type:"boolean"}
By_Size_Descending = False #@param {type:"boolean"}
By_Size_Ascending = "" #@param {type:"boolean"}
By_Time = "" #@param {type:"boolean"}

sort_seeders = "seeders/desc"
sort_size_asc = "size/asc"
sort_size_desc = "size/desc"
sort_time = "time/desc"

def scrap1337x(search, sort, page = 1):
  search = search.replace(" ","+").replace("'", " ")
  url = f"https://www.1337xx.to/sort-search/{search}/{sort}/{page}"
  print("Page: {} \n".format(url))
  response = requests.get(url, headers= {"Accept-Language": "en-US"})
  if response.status_code == 200 :  
    soup = BeautifulSoup(response.text, 'html.parser' )
    main_element = soup.find("tbody")
    if main_element is None:
      print("null search")
      return
    all_res = main_element.findAll("tr")
    title = "" 
    size = ""
    uploader = ""
    seeders = ""
    leechers = ""
    results = [] 
    if all_res:
      print("    {:<134}{:^12}    {:<15}{:>16}{:>14}\n".format("Title", "Size", "Uploader", "Seeders", "Leechers"))

    for i in  range(len(all_res)):
      alldata= all_res[i].findAll("td")
      title = alldata[0].findAll("a")[1].text
      size = alldata[4].text
      seeders = alldata[1].text
      leechers = alldata[2].text
      uploader = alldata[5].a.text
      print("{:<2}- {:<132}{:>11}       {:<16}{:>12}{:>13}".format(i + 1, title.strip(), size, uploader, seeders, leechers))
      href = "https://www.1337xx.to" + alldata[0].findAll("a")[1]['href']
      res = type('',(object,),{"title": title, "size" : size, "seeders" : seeders, "leechers" : leechers, "uploader" : uploader, "href" : href})()
      results.append(res)

    print("\nPlease choose a result to download:\n        Enter s or q to quit, n for the next page.\n")
    choice = input("")
    if choice == 'n':
      return 1

    if choice == "s" or choice == "q":
      print("\nExiting siege equipment..\n")
      return 0
    
    select= results[int(choice) - 1]
    print(f"\n Downloading file for and adding {select.title} :\n")
    url = select.href
    download = requests.get(url)
    soup = BeautifulSoup(download.text, "html.parser")
    hash = soup.find(class_="infohash-box").span.text.strip()                                
    download_link = f"https://itorrents.org/torrent/{hash}.torrent"
    file = "/content/{}.torrent".format(new_mv(select.title))
    !wget "$download_link" -O "$file" &> /dev/null
    
    add_local_files()
    print("Done.\n")
    return 0
  else:
    print(f" An error occured while trying to access 1337x.com  status: {response.status_code}")
    return 0

sort = sort_seeders
if By_Size_Descending:
  sort = sort_size_desc
elif By_Size_Ascending:
  sort = sort_size_asc
elif By_Time:
  sort = sort_time

if Search:
  state = 1
  while state != 0 :
    state = scrap1337x(Search, sort, Page)
    Page+=1

#### Auto download

In [ ]:
import requests, pprint
from bs4 import BeautifulSoup
import rarbgapi

#@markdown
Movie = True #@param {type: "boolean"}
TV_Show = False #@param {type:"boolean"}
Years="" #@param {type:"string"}
Ratings="7-" #@param {type:"string"}
Min_Votes="" #@param {type:"string"}
Max_Runtime_M="" #@param {type:"string"}
IMDB_Lists="ls091025400" #@param {type:"string"}
Top_250=False #@param {type:"boolean"}
Top_1000=False #@param {type:"boolean"}
Oscar_Winner=False #@param {type:"boolean"}
Best_Picture_Oscar=False #@param {type:"boolean"}
Best_Director_Oscar=False #@param {type:"boolean"}
English_Acted_Only=False #@param {type:"boolean"}
Animations=False #@param {type:"boolean"}
International_Movies=False #@param {type:"boolean"}
Indian_Movies=False #@param {type:"boolean"}
Sort_Date=True #@param {type:"boolean"}
Sort_Rating=False #@param {type:"boolean"}
Sort_Alphabatically=False #@param {type:"boolean"}
Sort_Ascending=False #@param {type:"boolean"}
Sort_Descending=True #@param {type:"boolean"}
Filter_Downloaded = False #@param {type:"boolean"}
Filter_Watched = False #@param {type:"boolean"}

#@markdown

l337x_Download = False #@param {type:"boolean"}
Rarbg_Download = False #@param {type:"boolean"}
#@markdown

Movie265_Category = True #@param {type:"boolean"}
Movie264_Category = False #@param {type:"boolean"}
TV_Category = False #@param {type:"boolean"}
TV_HD_Category = False #@param {type:"boolean"}

category_tv = rarbgapi.RarbgAPI.CATEGORY_TV_EPISODES  
category_tv_hd = rarbgapi.RarbgAPI.CATEGORY_TV_EPISODES_HD  
category_movie_h264 = rarbgapi.RarbgAPI.CATEGORY_MOVIE_H264_1080P  
category_movie_h265 = rarbgapi.RarbgAPI.CATEGORY_MOVIE_H265_1080P

exclude="%21"
international = "ls560608427"
indian= "ls560456074"
animation = "ls560333050"
watched_movies ="ls047519430"
watched_series = "ls047701980"
imdb = "https://www.imdb.com/title/"
date_sort = "release_date"
rating_sort = "user_rating"
alpha_sort = "alpha"

top250="top_250"
top1000= "top_1000"
oscar_winner="oscar_winner"
best_picture_oscar="best_picture_winner"
best_director_oscar="best_director_winner"

lists=""
imdb_labels = ""
start_year=""
end_year=""
ratings = Ratings.replace("-",",")
sort=""
if not '-' in Years:
  start_year = Years
  end_year= Years
else:
  vals = Years.split("-")
  start_year = vals[0]
  end_year= vals[1]

if Top_250:
  imdb_labels= imdb_labels + top250 + ","

if Top_1000:
  imdb_labels= imdb_labels + top1000 + ","

if Oscar_Winner:
  imdb_labels= imdb_labels + oscar_winner + ","

if Best_Picture_Oscar:
  imdb_labels= imdb_labels + best_picture_oscar + ","

if Best_Director_Oscar:
  imdb_labels= imdb_labels + best_director_oscar 

if "," in IMDB_Lists:
  for item in IMDB_Lists.split(","):
    lists = lists + item + ","

elif IMDB_Lists:
  lists = lists + IMDB_Lists + ","

if English_Acted_Only:
    lists = lists + exclude + international + "," + exclude + indian + "," + exclude + animation
else:
  if International_Movies:
    lists = international + ","

  if Indian_Movies:
    lists = lists + indian 

  if Animations:
    lists = lists + animation 

if Sort_Date:
  sort = date_sort

elif Sort_Rating:
  sort = rating_sort

elif Sort_Alphabatically:
  sort = alpha_sort

else:
  sort = date_sort

if Sort_Ascending and not Sort_Descending:
  sort = sort + ",asc"

else:
  sort = sort + ",desc"

rarbg = rarbgapi.RarbgAPI()
def DownloadRarbg(search):
  categories = []
  if Movie265_Category :
    categories.append(category_movie_h265)
  if Movie264_Category :
    categories.append(category_movie_h264)
  if TV_Category :
    categories.append(category_tv)
  if TV_HD_Category :
    categories.append(category_tv_hd)  

  print(f"Searching for:  {search}  in rarbg.com.\n")
  table = search.maketrans(":.","  ","(')?!")
  search = search.translate(table)
  results = rarbg.search(search_string=search, categories=categories)
  if not results:
    print("No results were found.\n")
    if not Movie264_Category:
      print("Trying to search in rarbg x264 category..\n")
      categories.append(category_movie_h264)
      results = rarbg.search(search_string=search, categories=categories)
    else:
      return
  if results:
    if len(results) == 1:
      print("Adding torrent: {}".format(results[0].filename))
      add_torrent_magnet(results[0].download, search)
      print("\n................\n")
      return
    else: 
      print("     {:<70}{:<10}{:<10}".format("Title", "", ""))
      for i in range(len(results)):
        print(" {}- {:<70} ".format(i + 1 , results[i].filename))

      print("\nPlease choose one of the results to download:             Format [single]     Enter s to skip\n")
      choice = input("")
      if choice == "s" or choice == "q":
        return
      choice = int(choice) - 1
      if choice < 0 or choice >= len(results):
        print("Choice out of range.\n")
        return
      print(f"\nAdding torrent: {results[choice].filename}\n")
      add_torrent_magnet(results[choice].download, search)
      print("   ...................")
    return
  else:
    print("No results were found.\n")
    return

def AdvancedMovieSearch (s_year, e_year, _ratings, min_votes, max_run, _lists, _sort):
  url=f'https://www.imdb.com/search/title/?title_type=feature&release_date={s_year}-01-01,{e_year}-12-31&user_rating={_ratings}&num_votes={min_votes},&runtime=,{max_run}&my_ratings=exclude&lists={_lists}&groups={imdb_labels}&sort={_sort}&view=simple&count=250'
  print()
  resp=requests.get(url, headers = {"Accept-Language": "en-US"})
  if resp.status_code==200:
      print("--> Search done successfully.")
      print()
      soup=BeautifulSoup(resp.text,'html.parser')    
      l=soup.find("div",{"class":"lister-list"})
      if not l :
        return
      movie_list = []
      downloaded= []
      all_items = l.findAll(class_="lister-item-header")
      shift = 0
      for i in range(len(all_items)):
        year = all_items[i].findChild(class_="lister-item-year")
        title = "{0} {1}".format(all_items[i].a.text, year.text).replace("(I) ", "").replace(": ", " ").replace(". ",".")
        table = title.maketrans("."," ","(,'?)")
        title = title.translate(table)
        if title in watched_movies_list and Filter_Watched:
          continue
        if movie_exists(title):
          downloaded.append(title)
          if Filter_Downloaded:
            continue         
        movie_list.append(title)
      print()
      watch_state=""
      if Filter_Downloaded:
        downloaded.sort()
        for item in downloaded:
          if item in watched_movies_list:
            watch_state = "WATCHED"
          else:
            watch_state = ""
          print("        {:<74}DOWNLOADED{:>16}".format(item, watch_state))
        #print()
        for index in range(len(movie_list)):
          if movie_list[index] in watched_movies_list:
            watch_state = "WATCHED"
          else:
            watch_state = ""
          print("  {:>3} - {:<75}{:>25}".format(index + 1 , movie_list[index], watch_state))
      else:
        watch_state = ""
        download_state=""
        for index in range(len(movie_list)):          
          if movie_list[index] in watched_movies_list:
            watch_state = "WATCHED"
          else:
            watch_state = ""          
          if movie_list[index] in downloaded:
            download_state = "DOWNLOADED"
          else:
            download_state = ""
          print("  {:>3} - {:<75}{:10}{:>16}".format(index + 1, movie_list[index], download_state, watch_state))
      
      if Rarbg_Download or l337x_Download:
        print("\n--> Choose movie range to download:       Format: [start-end]      [title1,title5..]     [single]      Enter [q] to exit\n")
        start= -1
        end = -1
        rng = input("")
        print()
        if rng.lower() == 'q':
          return
          
        elif "," in rng:
          rngs = rng.split(',')
          for item in rngs:
            if "-" in item:
              in_rngs = item.split('-')
              start = int(in_rngs[0])-1
              end = int((in_rngs[1]))
              if l337x_Download:
                for i in range(start, end):
                  scrap1337x(movie_list[i], sort_seeders)
              elif Rarbg_Download:    
                for i in range(start, end):
                  DownloadRarbg(movie_list[i])
            else:
              start = int(item)-1
              if l337x_Download:
                scrap1337x(movie_list[start], sort_seeders)
              elif Rarbg_Download:  
                DownloadRarbg(movie_list[start])  

        elif "-" in rng:
          rngs = rng.split('-')
          start = int(rngs[0])-1
          end = int((rngs[1]))
          if l337x_Download:
            for i in range(start, end):
              DownloadRarbg(movie_list[i])
          elif Rarbg_Download:  
            for i in range(start, end):
              DownloadRarbg(movie_list[i])

        else:
          start = int(rng) - 1
          if l337x_Download:
            scrap1337x(movie_list[start], sort_seeders)
          elif Rarbg_Download:  
            DownloadRarbg(movie_list[start]) 
              
        ses.start_dht()
        print("\n         Done")
  else:
      print("Error accessing the web page {0}".format(resp.state_code))

AdvancedMovieSearch(start_year, end_year,ratings,Min_Votes, Max_Runtime_M, lists, sort)



#### Add torrents

In [ ]:
#@markdown

from google.colab import files

Magnet_Torrent = False #@param {type: "boolean"}
Upload_Torrent_File = False #@param {type:"boolean"}
From_Local_File = True #@param {type:"boolean"}

#@markdown

Movie = False #@param {type: "boolean"}
TV_Show = False #@param {type:"boolean"}

if Upload_Torrent_File or From_Local_File:
  nms = []
  if Upload_Torrent_File:
    source = files.upload()
    nms = list(source.keys())
    print()
  else:
    add_local_files()

if Magnet_Torrent:
  magnet_link = ""
  while True:
    magnet = input("Enter magnet link or type q:  ")
    if magnet_link.lower() == "q":
      break
    name = get_magnet_name(magnet_link)
    if Movie and movie_exists(name):
      print(name + " is already downloaded.")
      continue
    elif TV_Show and show_exists(name):
      print(name + " is already downloaded.")
      continue
    add_torrent_magnet(magnet)   

ses.start_dht()
print('')
print("Save path: "+ drive_path)
print("Temp Save: " + temp_save)

#### Download metadata

In [ ]:

#@markdown 

#@markdown 

print("Downloading metadata")
checks=[]
for h in active:
  h.resume()
  checks.append(h)

while (checks):
  i = input("enter anything to exit  ")
  if i :
    break
    
  for h in checks:
    if h.has_metadata() :
      h.pause()
      checks.remove(h)
  time.sleep(1)

for h in active:
  if(h.has_metadata()):
    tor_info = h.get_torrent_info()
    files = tor_info.files()
    for x in range(files.num_files()):
      h.file_priority(x,5)

time.sleep(2)

#@markdown

total_download = 0

print("Download files:")
for h in active:

  print("")
  print("  "+h.name())

  if(h.has_metadata()):

    priorities = h.file_priorities()
    tor_info = h.get_torrent_info()
    files = tor_info.files()
    file_sizes = tor_info.total_size()
    total_download += file_sizes
    size_str = data_str(file_sizes)

    print("  Total download size: " + size_str)
    print("  Index Size    Name")

    fi_str=""
    for x in range(files.num_files()):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str
      print(fi_str,files.file_name(x))
  else:
    print("  Metadata not recieved yet.")
    
print("")
all_total_str = data_str(total_download)
print("Total download: ", all_total_str)


#### Filter files

In [ ]:
#@markdown 

#@markdown 

skip_list = ['RARBG.txt',
             'RARBG_DO_NOT_MIRROR.exe',
             'www.YTS.LT.jpg',
             '.txt',
              '.inf'
             ]

for h in active:
  if(h.has_metadata()):

    torinfo = h.get_torrent_info()
    files = torinfo.files()
    for x in range(files.num_files()):
      if any(i in files.file_name(x) for i in skip_list):
        h.file_priority(x,0)
      else: 
        h.file_priority(x,5)

#@markdown

time.sleep(2)
total_download = 0

print("Download files:")
for h in active:

  print("")
  print("  "+h.name())

  if(h.has_metadata()):

    priorities = h.file_priorities()
    torinfo = h.get_torrent_info()
    files = torinfo.files()
    status= h.status()
    file_sizes = status.total_wanted
    total_download += file_sizes

    size_str = data_str(file_sizes)

    print("  Total download size: " + size_str)
    print("  Index Size    Name")

    fi_str=""
    for x in range(files.num_files()):
      if (priorities[x]>0):
        fs_str= data_str(files.file_size(x))
        fi_str = "   {:3d}-  ".format(x) + fs_str
        print(fi_str,files.file_name(x))
  else:
    print("  Metadata not recieved yet.")
    
print("")
all_total_str = data_str(total_download)
print("Total download: ", all_total_str)

#### Add single torrent magnet

In [ ]:
#@markdown

import time

ses.resume()

params = {"save_path": temp_save}

#@markdown

#@markdown Enter magnet link
MagnetLink = "" #@param {type:"string"}

handle = lt.add_magnet_uri(ses, MagnetLink, params)
#from traitlets.traitlets import SequenceTypes

#@markdown

#@markdown 

Sequential_Download = False #@param{type:"boolean"}
#AutoManaged = False  #@param{type:"boolean"}
#handle.auto_managed(AutoManaged)

if Sequential_Download:
  handle.set_sequential_download(True)  

handle.auto_managed(True)

if handle in completed or handle in active:
  print("Torrent already added.")

else :
  active.append(handle)
  ses.start_dht()

  print("Save path: "+drive_path)
  print("Temp Save: " + temp_save)
  print('')

  i=''
  while not (handle.has_metadata() and i):
    time.sleep(0.5)
    i=input("Enter anything to exit: ")

  handle.pause()
  status = handle.status()
  print("Sequential: "+str(status.sequential_download))
  print('')

  if(handle.has_metadata()):
    torinfo = handle.get_torrent_info()
    files = torinfo.files()

    for x in range(files.num_files()):
      handle.file_priority(x,5)

    file_sizes = torinfo.total_size()
    size_str = data_str(file_sizes)

    print("Total download size: "+size_str)
    print("Index Size    Name")
    print('')

    fi_str=""
    for x in range(files.num_files()):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str
      print(fi_str,files.file_name(x))

  else:
    print("Metadata not recieved yet.")

#### Choose seasons

In [ ]:
s_list=["S01","S02","S03","S04","S05","S06","S07","S08","S09","S10","S11","S12"]
check_list =["False","False","False","False","False","False","False","False","False","False","False","False"]
ok_list = [ ]
handle = active[0]

S01=False #@param {type:"boolean"}
S02=False #@param {type:"boolean"}
S03=False #@param {type:"boolean"}
S04=False #@param {type:"boolean"}
S05=False #@param {type:"boolean"}
S06=False #@param {type:"boolean"}
S07=False #@param {type:"boolean"}
S08=False #@param {type:"boolean"}
S09=False #@param {type:"boolean"}
S10=False #@param {type:"boolean"}
S11=False #@param {type:"boolean"}
S12=False #@param {type:"boolean"}

check_list[0]=S01
check_list[1]=S02
check_list[2]=S03
check_list[3]=S04
check_list[4]=S05
check_list[5]=S06
check_list[6]=S07
check_list[7]=S08
check_list[8]=S09
check_list[9]=S10
check_list[10]=S11
check_list[11]=S12

for index, season in enumerate(s_list[:]):
  if check_list[index]:
    ok_list.append(season)


if(handle.has_metadata()):
  torinfo = handle.get_torrent_info()
  files = torinfo.files()

  for x in range(files.num_files()):
    if any(i in files.file_name(x) for i in ok_list):
      handle.file_priority(x,5)
    else:
      handle.file_priority(x,0)

  time.sleep(2)

  total_download = 0

  print("Download files:")
  print("")

  print("  "+handle.name())

  priorities = handle.file_priorities()
  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  status= handle.status()
  file_sizes = status.total_wanted

  size_str = data_str(file_sizes)
  print("   Total download size: " + size_str)
  print("   Index Size    Name")

  fi_str=""
  for x in range(files.num_files()):
    if (priorities[x]>0):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str
      print(fi_str,files.file_name(x))

      
  print("")
  print("")

  print("  Total download: ", size_str)

else:
  print("    Metadata not recieved yet.")


#### Choose ranges

In [ ]:
#@markdown Choose files ranges

#@markdown 

handle = active[0]

Start=0 #@param {type:"integer"}
End=0 #@param {type:"integer"}

if(handle.has_metadata()):
  torinfo = handle.get_torrent_info()
  files = torinfo.files()

  for index in range(len(priorities)):
    if index >= Start and index <= End:
      handle.file_priority(index,5)
    else:
      handle.file_priority(index,0)

  time.sleep(2)

  total_download = 0

  print("Download files:")
  print("")

  print("  "+handle.name())

  priorities = handle.file_priorities()
  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  status= handle.status()
  file_sizes = status.total_wanted

  size_str = data_str(file_sizes)
  print("   Total download size: " + size_str)
  print("   Index Size    Name")

  fi_str=""
  for x in range(files.num_files()):
    if (priorities[x]>0):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str

      print(fi_str,files.file_name(x))

      
  print("")
  print("")

  print("  Total download: ", size_str)

else:
  print("   Metadata not recieved yet.")


In [ ]:
#@markdown Addtional Ranges

handle = active[0]

Second_Range=False #@param {type:"boolean"}
Range1_Start=0 #@param {type:"integer"}
Range1_End=0 #@param {type:"integer"}

#@markdown 

#@markdown 

Third_Range=False #@param {type:"boolean"}
Range2_Start=0 #@param {type:"integer"}
Range2_End=0 #@param {type:"integer"}

#@markdown 

#@markdown 

Exlude_Range=False #@param {type:"boolean"}
Exlude_Range_Start=0 #@param {type:"integer"}
Exlude_Range_End=0 #@param {type:"integer"}


if(handle.has_metadata()):
  file_count = len(handle.file_priorities())
  torinfo = handle.get_torrent_info()
  files = torinfo.files()

  for index in range(file_count):
    if Second_Range and (index >= Range1_Start and index <= Range1_End):
      handle.file_priority(index,5)

    if Third_Range and (index >= Range2_Start and index <= Range2_End):
      handle.file_priority(index,5)

    if Exlude_Range and (index >= Exlude_Range_Start and index <= Exlude_Range_End):
      handle.file_priority(index,0)

  time.sleep(2)

  total_download = 0

  print("Download files:")
  print("")

  print("  "+handle.name())

  priorities = handle.file_priorities()
  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  status= handle.status()
  
  file_sizes = status.total_wanted

  size_str = data_str(file_sizes)
  print("   Total download size: " + size_str)
  print("   Index Size    Name")

  fi_str=""
  for x in range(files.num_files()):
    if (priorities[x]>0):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str

      print(fi_str,files.file_name(x))

  print("")
  print("")

  print("  Total download: ", size_str)

else:
  print("   Metadata not recieved yet.")

#### Check choosen files

In [ ]:

#@markdown

#@markdown


total_download = 0

count_str = str(len(active))

print("Downloads: {0}".format(count_str))
for h in active:
  print("")
  print("  "+h.name())

  if(h.has_metadata()):
    priorities = h.file_priorities()
    torinfo = h.get_torrent_info()
    files = torinfo.files()
    status= h.status()
    file_sizes = status.total_wanted
    total_download+=file_sizes

    size_str = data_str(file_sizes)

    print("   Total download size: " + size_str)
    print("   Index Size    Name")

    fi_str=""
    for x in range(len(priorities)):
      if (priorities[x]>0):
        fs_str= data_str(files.file_size(x))
        fi_str = "   {:3d}-  ".format(x) + fs_str

        print(fi_str,files.file_name(x))
  else:
    print("    Metadata not recieved yet.")
    
print("")
all_total_str = data_str(total_download)
print("Total download: ", all_total_str)

#### Add trackers

In [ ]:
#@markdown 

#@markdown 
#@markdown 

#@markdown 

new_trackers = ["udp://tracker.opentrackr.org:1337/announce"
,"udp://tracker.internetwarriors.net:1337/announce"
,"udp://exodus.desync.com:6969/announce"
,"udp://tracker.cyberia.is:6969/announce"
,"udp://3rt.tace.ru:60889/announce"
,"http://5rt.tace.ru:60889/announce"
,"udp://explodie.org:6969/announce"
,"udp://47.ip-51-68-199.eu:6969/announce"
,"udp://opentracker.i2p.rocks:6969/announce"
,"http://open.acgnxtracker.com:80/announce"
,"udp://www.torrent.eu.org:451/announce"
,"udp://tracker.torrent.eu.org:451/announce"
,"udp://tracker.tiny-vps.com:6969/announce"
,"udp://tracker.ds.is:6969/announce"
,"udp://retracker.lanta-net.ru:2710/announce"
,"udp://open.stealth.si:80/announce"
,"udp://tracker.moeking.me:6969/announce"
,"udp://ipv4.tracker.harry.lu:80/announce"
,"http://tracker.dler.org:6969/announce"
,"http://rt.tace.ru:80/announce"
,"udp://93.158.213.92:1337/announce"
,"udp://138.255.103.83:1337/announce"
,"udp://208.83.20.20:6969/announce"
,"udp://193.218.118.220:6969/announce"
,"udp://79.111.118.59:60889/announce"
,"http://93.157.234.32:60889/announce"
,"udp://184.105.151.164:6969/announce"
,"udp://51.68.199.47:6969/announce"
,"udp://51.81.46.170:6969/announce"
,"udp://91.216.110.52:451/announce"
,"udp://89.234.156.205:451/announce"
,"udp://5.206.60.196:6969/announce"
,"udp://5.226.148.20:6969/announce"
,"udp://37.235.174.46:2710/announce"
,"udp://185.181.60.67:80/announce"
,"udp://138.201.150.56:6969/announce"
,"udp://51.15.40.114:80/announce"
,"http://168.119.237.9:6969/announce"
,"http://195.201.31.194:80/announce"
,"https://trakx.herokuapp.com:443/announce"
,"https://w.wwwww.wtf:443/announce"
,"https://tracker.tamersunion.org:443/announce"
,"https://tracker.sloppyta.co:443/announce"
,"https://tracker.renfei.net:443/announce"
,"https://tracker.nitrix.me:443/announce"
,"https://tracker.nanoha.org:443/announce"
,"https://tracker.lelux.fi:443/announce"
,"https://tracker.imgoingto.icu:443/announce"
,"https://tracker.hama3.net:443/announce"
,"https://tracker.foreverpirates.co:443/announce"
,"http://tracker2.wasabii.com.tw:6969/announce"
,"udp://tracker.sktorrent.net:6969/announce"
,"http://www.wareztorrent.com:80/announce"
,"udp://bt.xxx-tracker.com:2710/announce"
,"udp://tracker.eddie4.nl:6969/announce"
,"udp://tracker.grepler.com:6969/announce"
,"udp://tracker.mg64.net:2710/announce"
,"udp://wambo.club:1337/announce"
,"udp://tracker.dutchtracking.com:6969/announce"
,"udp://tc.animereactor.ru:8082/announce"
,"udp://tracker.justseed.it:1337/announce"
,"udp://tracker.leechers-paradise.org:6969/announce"
,"https://open.kickasstracker.com:443/announce"
,"udp://tracker.coppersurfer.tk:6969/announce"
,"http://87.253.152.137/announce"
,"http://91.216.110.47/announce"
,"http://91.217.91.21:3218/announce"
,"http://91.218.230.81:6969/announce"
,"http://93.92.64.5/announce"
,"http://atrack.pow7.com/announce"
,"http://bt.henbt.com:2710/announce"
,"http://bt.pusacg.org:8080/announce"
,"https://tracker.bt-hash.com:443/announce"
,"udp://tracker.leechers-paradise.org:6969"
,"https://182.176.139.129:6969/announce"
,"udp://zephir.monocul.us:6969/announce"
,"https://tracker.dutchtracking.com:80/announce"
,"https://grifon.info:80/announce"
,"udp://tracker.kicks-ass.net:80/announce"
,"udp://p4p.arenabg.com:1337/announce"
,"udp://tracker.aletorrenty.pl:2710/announce"
,"https://tracker.parrotsec.org:443/announce"
,"https://tracker.moxing.party:6969/announce"
,"https://tracker.ipv6tracker.ru:80/announce"
,"https://tracker.fastdownload.xyz:443/announce"
,"https://gwp2-v19.rinet.ru:80/announce"
,"https://tr.kxmp.cf:80/announce"
,"https://explodie.org:6969/announce"]


for h in active:
  trackers = h.trackers()
  #last_tier = trackers[-1]['tier']
  tier = 3
  for x in range(len(new_trackers)):
    annouce_entry = {'url': new_trackers[x], 'tier': tier}
    h.add_tracker(annouce_entry) 

#### List handle trackers

In [ ]:
#@markdown 

#@markdown 

trackers = handle.trackers()
print("Trackers count ",len(trackers))

trackers = handle.trackers()
print("Trackers count ",len(trackers))

for x in trackers:
  #print(x)
  print(x['url'], x['tier'])

#### Start Download

In [ ]:
#@markdown

import time, copy
from IPython.display import display
import ipywidgets as w

downloads= ses.get_torrents()

state_str = ["Queued","Checking","Downloading metadata","Downloading","Finished","Seeding","Allocating","Checking resume files"]

progress_labels= []
progress_bars = []
states = []
download_speeds =[]
upload_speeds =[]
total_downloads =[]
total_uploads =[]

style0= {"padding-left":"200px","padding-right":"200px"}
layout_15 = w.Layout(width='auto', flex='15 1 0%', align_items="center")
layout_10 = w.Layout(width='auto', flex='10 1 0%', align_items="center")
layout_8 = w.Layout(width='auto', flex='8 1 0%', align_items="center")
layout_7 = w.Layout(width='auto', flex='7 1 0%', align_items="center")
layout_5 = w.Layout(width='auto', flex='5 1 0%', align_items="center")
layout_4 = w.Layout(width='auto', flex='4 1 0%', align_items="center")
layout_3 = w.Layout(width='auto', flex='3 1 0%', align_items="center")
layout_2 = w.Layout(width='auto', flex='2 1 0%', align_items="center")
layout_1 = w.Layout(width='auto', flex='1 1 0%', align_items="center")
layout_1E = w.Layout(width='auto', flex='1 1 0%', align_items="center")

layout_up = {'flex_flow':'row','align_items':'center','width':'100%','justify_content':'space-around'}
layout_r = {'padding-left':'100px','flex_flow':'row','align_items':'flex-start','width':'95%','justify_content':'space-around'}
layout_c = {'width':'95%','align_items':'center','border':'solid #d9dbda 1px'}
layout_v = {'flex_flow':'row','align_items':'center','width':'95%','justify_content':'space-around'}

exit_btn = w.ToggleButton(
    disabled=False,
    button_style="danger",
    icon="check",
    value=False,
    layout=layout_1
)

empty_label_1= w.Label(value="",layout=layout_1)
empty_label_2= w.Label(value="",layout=layout_2)
empty_label_3= w.Label(value="",layout=layout_3)
empty_label_4= w.Label(value="",layout=layout_4)
empty_label_5= w.Label(value="",layout=layout_5)

total_torrents_label= w.Label(value="Total torrents:  {0}".format(len(downloads)),layout=layout_3)
active_label= w.Label(value="Active:  {0}".format(len(active)),layout=layout_3)
completed_label= w.Label(value="Completed:  {0}".format(len(completed)),layout=layout_3)

total_download_label= w.Label(value="Session download:  {0}".format(data_str(0)),layout=layout_5)
total_progress_label= w.Label(value="{0}%".format(0),layout=layout_3)
total_upload_label= w.Label(value="Session upload:  {0}".format(data_str(0)),layout=layout_5)

def Download_View(handle):
  status = handle.status()
  progress_bar = w.FloatProgress(
      value= status.progress*100,
      min=0,
      max=100,
      step=0.5,
      bar_style='info',
      orientation='horizontal',
      layout=layout_15
  )
  download_label = w.Label(value=handle.name(), layout = layout_10)
  progress_label= w.Label(value="{:.2f}%".format(status.progress*100), layout=layout_1)
  progress_bars.append(progress_bar)
  progress_labels.append(progress_label)

  strs=[
        state_str[status.state],
        "Download Speed:    {0}".format(speed_str(status.download_rate)),
        "Upload speed:    {0}".format(speed_str(status.upload_rate)),
        "Total Download:    {0} / {1}".format(data_str(status.total_wanted_done),data_str(status.total_wanted)),
        "Total Upload:    {0}".format(data_str(status.total_upload)) #all_time_upload not working
        ]

  ls = [w.Label(value=v,layout=layout_4) for v in strs]

  states.append(ls[0])
  download_speeds.append(ls[1])
  upload_speeds.append(ls[2])
  total_downloads.append(ls[3])
  total_uploads.append(ls[4])

  ls.insert(5,empty_label_3)

  r1 = w.Box([ download_label,progress_label, progress_bar,empty_label_1],layout=layout_r,style=style0)
  r2 = w.Box(ls,layout=layout_r,style=style0)
  c = w.VBox([r1,r2],layout=layout_c)
  v = w.Box([c, empty_label_1],layout=layout_r,style=style0)

  return c

def Complete_View(handle):
  status = handle.status()
  progress_bar = w.FloatProgress(
      value= 100,
      min=0,
      max=100,
      step=0.1,
      bar_style='success',
      orientation='horizontal',
      layout=layout_15
  )
  download_label = w.Label(value = handle.name(), layout = layout_10)
  progress_label= w.Label(value = "100%",layout = layout_1E)

  upload = status.all_time_upload
  if upload == 0:
    upload = status.total_upload

  strs=[
        "Completed",
        "Completed at:    {0}".format(time.strftime('%I:%M:%S %p', time.localtime(status.completed_time))),
        "Total time:     {:.2f} m".format(status.active_time/60),
        "Download:    {0} / {1}".format(data_str(status.total_wanted_done),data_str(status.total_wanted)),
        "Upload:    {0}".format(data_str(upload))
        ]

  ls = [w.Label(value=v,layout=layout_4) for v in strs]

  ls.insert(5,empty_label_3)

  r1 = w.Box([ download_label,progress_label, progress_bar,empty_label_1],layout=layout_r,style=style0)
  r2 = w.Box(ls,layout=layout_r,style=style0)
  c = w.VBox([r1,r2],layout=layout_c)
  return c

def Update_View(i, status):
  states[i].value = state_str[status.state]
  progress_labels[i].value =  "{:.2f}%".format(status.progress*100)
  progress_bars[i].value = status.progress*100
  download_speeds[i].value= "Download Speed:    {0}".format(speed_str(status.download_rate))
  upload_speeds[i].value= "Upload speed:    {0}".format(speed_str(status.upload_rate))
  total_downloads[i].value= "Download:    {0} / {1}".format(data_str(status.total_wanted_done),data_str(status.total_wanted))
  total_uploads[i].value= "Upload:    {0}".format(data_str(status.total_upload))
  return

total_download = 0
total_wanted = 0
total_up = 0
completed_down = 0
completed_up = 0
total_progress = 0

def Update_Stats(_download,_upload):
  total_progress = (100*_download)/total_wanted
  active_label.value = "Active:  {0}".format(len(active))
  completed_label.value = "Completed:  {0}".format(len(completed))
  total_download_label.value = f"Session download:   {data_str(_download)} / {data_str(total_wanted)}"
  total_progress_label.value = f"     {round(total_progress,2)}%"
  total_upload_label.value = "Session upload:   {}".format(data_str(_upload))

def Remove_Active_View(i):
  active_views[i].layout.display = 'none'
  del active_views[i]
  del progress_labels[i]
  del progress_bars[i]
  del download_speeds[i]
  del upload_speeds[i]
  del total_downloads[i]
  del total_uploads[i]
  return


def Completed_Stats():
  _download=0
  _upload=0
  for c in completed:
    status = c.status()
    upload = status.all_time_upload
    if upload == 0:
      upload = status.total_upload  

    _download+= status.total_wanted_done
    _upload+= upload
  return [_download,_upload]

ses.resume()
active.clear() 
completed_views = []
active_views = []
for handle in downloads:
  status = handle.status()
  total_wanted+=status.total_wanted
  if handle in completed:
    completed_views.append(Complete_View(handle))
    continue
  if handle not in active:
    active.append(handle)
  active_views.append(Download_View(handle))
  handle.resume()
    
if Movie:
  save_path = Movie_Path
elif TV_Show:
  save_path = Shows_Path
else:
  save_path = drive_path

active_down = 0 
active_up = 0 
stat= Completed_Stats()
completed_down=stat[0]
completed_up=stat[1]

Update_Stats(completed_down, completed_up)

stats_row = w.Box([total_torrents_label, active_label, completed_label, total_download_label, total_progress_label, total_upload_label, empty_label_3],layout=layout_up,style=style0)
exit_row = w.Box([ empty_label_1, exit_btn],layout=layout_r,style=style0)
session_state = "SESSION IS RUNNING                  Save to: " + save_path + "              Temp save: " + temp_save
print()
print()
print()
print()
print("                                    " + session_state)
print()
display(stats_row)
print()
print()
display(*active_views)
print()
print()
display(*completed_views)

while active:
  if exit_btn.value:
    print(exit_btn.value)
    break
  shift = 0
  active_down = 0
  active_up = 0
  active_wanted = 0
  for index, handle in enumerate(active[:]):    
    status = handle.status()
    if handle.is_finished():
      handle.pause()
      shave (handle, save_path)
      name = handle.name()
      new_name = ""
      if Movie:
        new_name = new_mv(name)
        path = status.save_path + "/" + name
        after_shave( path , new_name)
      elif TV_Show:
        new_name = new_tv(name)
      p1 = status.save_path + "/" + name
      p2 = done_path + "/" + new_name
      !mv "$p1" "$p2"
      if not new_name:
        new_name = "/" + name
      p3 = done_path + "/" + new_name 
      !mv "$p3" "$save_path"

      Remove_Active_View(index-shift)
      completed.append(handle)
      active.remove(handle)
      complete_view = Complete_View(handle)
      display(complete_view)
      completd_stat= Completed_Stats()
      completed_down=completd_stat[0]
      completed_up=completd_stat[1]
      shift= shift + 1
    else:
      Update_View(index-shift, status)
      active_up+=status.total_upload
      active_down+=status.total_wanted_done
      active_wanted+=status.total_wanted

  total_wanted = completed_down + active_wanted
  total_download = completed_down + active_down
  total_up = active_up + completed_up
  Update_Stats(total_download, total_up)
  time.sleep(0.9)

#### Pause session

In [ ]:
#@markdown 

#@markdown 

downloads= ses.get_torrents()
for h in downloads:
  try:
    print(h.name())
    h.pause()
  except:
    print("Invailed torrent handle.")
    
ses.pause()

#### Save download progress

In [ ]:
#@markdown 

Save_To = "/content/drive/Shareddrives/SM1/Downloads/.uncomplete" #@param {type: "string"}

def sv():
  if active:
    print(" Choose a download to save progress , q to exit:")
    for i in range(len(active)):
      print("   {0}-  {1}".format(i+1, active[i].name()))

    print()

    inpt =  input("")
    if inpt == 'q':
      return

    choice = int(inpt) - 1

    print()
    if(choice >= len(active) or choice < 0):
      print("The choice is out of range.")
    else:
      tr_name = active[choice].name()
      print("Saving {0} into {1}".format(tr_name, Save_To))
      print()
      path = temp_save +"/"+ tr_name
      !cp -r "$path" "$Save_To"
      print("..................")
      print("       Done")

sv()

#### Verify saving to drive

In [ ]:
#@markdown

downloads= ses.get_torrents()
not_copied=[]
for torrent in downloads:
    name = torrent.name()
    print("  Checking: {0}".format(name) )
    status = torrent.status()
    byte_check = status.total_wanted - status.total_wanted_done
    if download.is_finished() or byte_check == 0:
        copy_status = ""
        if movie_exists(name):
            copy_status ="copied"
        else:
            copy_status = "not copied"
            not_copied.append(torrent)
        print("  Status:   completed   {0}".format(copy_status))
    else:
        print("  Status:   not completed   state: {0}     progress: {1}%     downloaded: {2}     remained: {3}".format(status.state, round(status.progress*100,2), data_str(status.total_wanted_done), data_str(status.total_wanted-status.total_wanted_done)))
    print()
print(  "Downloads completed but not copied:")
if not not_copied:
    print("     None")
else:
    for index in range(len(not_copied)):
        print("     {0}- {1}".format( index + 1, not_copied[index].name()))
print()
if not_copied:
  print("Do you want to copy and complete saving these downloads? y/n")
  print()
  choice = input("")
  if choice == 'y':
    for handle in not_copied:
      new_name = ""
      if Movie:
        new_name = new_mv(name)
      elif TV_Show:
        new_name = new_tv(name)
        status = handle.status()
      p1 = status.save_path + "/" +  handle.name()
      p2 = done_path + "/" + new_name
      !mv "$p1" "$p2"
      if not new_name:
        p2 = p2 + handle.name()
      !cp -r "$p2" "$save_path"
      print(".................")
      print("       Done")


#### Clear Completed

In [ ]:
#@markdown

for download in completed:
  print("Deleting: " + download.name())
  ses.remove_torrent(download, 0)

completed.clear()


#### Clear completed in downloads

In [ ]:
#@markdown

for f in active:
  status= f.status()
  if status.is_finished() or status.is_seed():
    f.pause()
    active.remove(f)    
    if not f in completed:
      completed.append(f)


#### Delete all completed 

In [ ]:
#@markdown 

for f in completed:
  try:
    ses.remove_torrent(f)
  except:
    print("Invild torrent.")
    continue

completed.clear()

#@markdown 


#### Clear active

In [ ]:
#@markdown 

for f in active:
  f.pause()
  active.remove(f)
  try:
    ses.remove_torrent(f)
  except:
    print("Invalid torrent.")
  
active.clear()

#### Copy and move files

In [ ]:
#@markdown 

From_Path = "/content/drive/MyDrive/Library/Movies" #@param {type: "string"}
To_Path = "/content/drive/Shareddrives/SM1/Library/Movies" #@param {type: "string"}

Move = False #@param {type:"boolean"}
Rename_only = False #@param {type:"boolean"}
Movie_Name = True #@param {type:"boolean"}
Show_Name = False #@param {type:"boolean"}

def cp_mv(content):
  if content:
    print()
    print(" Enter directory range to copy [Format: start-end  All: . Exit: q]:")
    print_files(content, 0 ,len(content))
    print()
    start = -1
    end = -1
    rng = input("")
    print()
    if rng.lower() == 'q':
      return
    elif rng == '.':
      start = 0
      end = len(content)
    elif not '-' in rng:
      end = int(rng)
      start = end - 1
    else:
      rngs = rng.split('-')
      start = int(rngs[0])-1
      end = int((rngs[1]))
    print("Chosen range start: {0}, end: {1}".format(start+1,end))
    print()
    if end > len(content) or start < 0 or ((start + end) < 0) :
      print("The ranges' ends are out of the directory file range.")
      return
    elif Rename_only:
      print("Renaming these files")
      print()
      print_files(content,start,end)
      print()
      for i in range(start, end):
        name  = ""
        if Movie_Name:
          name = new_mv(content[i])
        elif Show_Name:
          name = new_tv(content[i])
        else:
          print("  Enter a new name:")
          name = input("")
        if not name:
          continue
        print("   Renaming   {0}   new name:   {1}".format(content[i], name))
        p1 = From_Path + "/" + content[i]
        p2 = From_Path + "/" + name 
        !mv "$p1" "$p2"

    elif Move:
      tp= From_Path + "/.moved"
      os.makedirs(tp, exist_ok = True)
      print("Moving these files to {0}".format(To_Path))
      print()
      print_files(content,start,end)
      print()
      for i in range(start, end):
        rename  = ""
        if Movie_Name:
          rename = "/" + new_mv(content[i])
        elif Show_Name:
          rename =  "/" + new_tv(content[i])

        print("   Moving  {0}".format(content[i]))

        p1 = From_Path + "/" + content[i]
        p2 = tp +  rename

        !mv "$p1" "$p2"
        if not rename:
          p2 = p2 + content[i]
        !cp -r "$p2" "$To_Path"
    
    else:
      print("Copying these files to {0}".format(To_Path))
      print()
      print_files(content,start,end)
      print()
      for i in range(start, end):
        rename  = ""
        if Movie_Name:
          rename = "/" + new_mv(content[i])
        elif Show_Name:
          rename =  "/" + new_tv(content[i])    

        p0 = To_Path + content[0]
        p1 = From_Path + "/" + content[i]
        p2 = To_Path + rename

        if os.path.exists(p0) or (rename and os.path.exists(p2)) :
          print("   Skipping:  {0}  directory alraedy exists.".format(content[i]))
          continue

        print("   Copying:  {0}".format(content[i]))
        !cp -r "$p1" "$p2"

    print()
    print("   .....................")
    print("           Done")
    return
  else:
    print("Empty directory.")
    return

drfiles = filter_dirs(next(os.walk(From_Path))[1])
drfiles.sort()
cp_mv(drfiles)

#### Rm

In [ ]:
#@markdown 

Delation_Path = "/content/drive/MyDrive/Library/Movies" #@param {type: "string"}


def rm(content):
  if content:
    print()
    print(" Enter directory range to copy (start-end):")
    print_files(content, 0 ,len(content))
    print()
    start = -1
    end = -1
    rng = input("")
    print()
    if rng == 'q':
      return
    if not '-' in rng:
      start = int(rng)
    else:
      rngs = rng.split('-')
      start = int(rngs[0])-1
      end = int((rngs[1]))
    print("Chosen range start: {0}, end: {1}".format(start+1,end))
    print()
    if end >= len(content) or start < 0 or ((start + end) < 0) :
      print("The ranges' ends are out of the directory file range.")
      return
    else:
      print("Deleting these directories".format(To_Path))
      print()
      print_files(content,start,end)
      print()
      for i in range(start, end):
        print("   Deleting  {0}".format(content[i]))
        path = Delation_Path+"/"+content[i]
        !rm -r "$path"

    print()
    print("   ......................")
    print("           Done")
    return
  else:
    print("Empty directory.")
    return

drfiles = filter_dirs(next(os.walk(Delation_Path))[1])
drfiles.sort()
rm(drfiles)

#### Clear temp

In [ ]:
#@markdown 
Delete_torrent_files = True #@param {type:"boolean"}
Clear_done_folder = True #@param {type:"boolean"}
Clear_temp_folder = False #@param {type:"boolean"}

if Delete_torrent_files:
  !find . -maxdepth 1 -name "*.torrent" -type f -delete 

if Clear_done_folder:  
  !rm -r "/content/done/"*

if Clear_temp_folder:
  !rm -r "$temp_save/"*  

#### Check general status

In [ ]:
#@markdown 

#@markdown 

from IPython.display import display
import ipywidgets as widgets

downloads= ses.get_torrents()

session_state = ""
if ses.is_paused() :
  session_state = "SESSION IS PAUSED"
else:
  session_state = "SESSION IS RUNNING"

print("            " + session_state)

layout = {'width':'auto', 'margin-left':'50px'}
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

for index, download in enumerate(downloads[:]):
    bar = download_bars[index]
    shift = 0
    try:
      status = download.status()
      state = ""
      if status.paused == True:
        state = "PAUSED"
      else:
        state_str[status.state]
      bar.description = " ".join(
          [
              download.name(),
              speed_str(ByteToMB(status.download_rate)),
              state,
              "{0} / {1}".format(status.total_wanted_done, status.total_wanted)
          ]
      )
      bar.value = round(status.progress * 100,2)
    except:
      downloads.remove(download)
      download_bars.remove(index-shift)
      shift = shift + 1
      print("Invalid torrent")
      continue


#### Resume all

In [ ]:
#@markdown 

ses.resume()

downloads= ses.get_torrents()

for h in downloads:
  h.resume()

#### Remove torrent

In [ ]:
#@markdown

downloads= ses.get_torrents()

print("Please select a torrent to remove:")
print("")

for index,h in enumerate(downloads[:]):
  print("{0}- {1}".format(index, h.name()))

print("")
select = int(input("Enter torrent selection:  "))
print("")

h = downloads[select]
print("")

print("Deleting {0}".format(h.name()))
print("...........")
print("")

h.pause()

if h in active:
  active.remove(h)

if h in completed:
  active.remove(h)

try:
  ses.remove_torrent(h)
  
except:
  print("Selected torrent is invalid.")  


print("Done.")

#### Clear downloads

In [ ]:
#@markdown 

#@markdown 

downloads= ses.get_torrents()

for download in downloads:
    print("Deleting: " + download.name())
    ses.remove_torrent(download, 0)

active.clear()
completed.clear()

#### Clean movies dir

In [ ]:
#@markdown

def clean_movies_dir() :
    dir_content = next(os.walk(Movie_Path))[1]
    for dir in dir_content :
        print(dir)
        newname = dir
        if "." in dir or "1080" in dir:
            newname = new_mv(dir)
            p1 = Movie_Path + "/" + dir
            p2 = Movie_Path + "/" + newname
            !mv "$p1" "$p2"
            
        path = Movie_Path + "/" + newname
        after_shave(path, newname)

clean_movies_dir()